In [ ]:
from pathlib import Path
import rasterio
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
!pip install rio_cogeo

In [ ]:
def convert_to_cog(input_tiff, output_tiff):
    """
    Convert a GeoTIFF to Cloud Optimized GeoTIFF (COG) in Web Mercator
    """
    # First create a temporary reprojected file
    temp_file = output_tiff.parent / 'temp_reprojected.tif'
    
    # Reproject to Web Mercator
    with rasterio.open(str(input_tiff)) as src:
        transform, width, height = calculate_default_transform(
            src.crs, 
            'EPSG:3857',
            src.width, 
            src.height, 
            *src.bounds
        )
        
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': 'EPSG:3857',
            'transform': transform,
            'width': width,
            'height': height
        })
        
        with rasterio.open(str(temp_file), 'w', **kwargs) as dst:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs='EPSG:3857',
                resampling=Resampling.bilinear
            )
    
    # Now create the COG from the reprojected file
    output_profile = cog_profiles.get("deflate")
    output_profile.update(
        tiled=True,
        blocksize=256,
        compress='deflate',
        predictor=2,
        overview_resampling='average'
    )
    
    cog_translate(
        str(temp_file),
        str(output_tiff),
        output_profile,
        quiet=False,
        allow_intermediate_compression=True,
        web_optimized=True
    )
    
    # Clean up temporary file
    temp_file.unlink()
    
    # Verify the output
    print("\nVerifying output COG...")
    with rasterio.open(str(output_tiff)) as dst:
        print(f"Output CRS: {dst.crs}")
        print(f"Output bounds: {dst.bounds}")

In [ ]:
data_dir = Path('../_data')
processed_folder = (data_dir / 'processed').resolve()
input_file = Path('../_data/raw/ST_Clipped_20190922.tif')
output_file = processed_folder / 'ST_Clipped_20190922_cog.tif'

convert_to_cog(input_file, output_file)